In [663]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import tensorflow as tf
import xgboost as xgb
import warnings
from tensorflow.keras.layers import Dense, LSTM, Conv1D, MaxPooling1D, TimeDistributed, Flatten, Dropout, RepeatVector
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller, kpss, ccf, pacf
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, TimeSeriesSplit
from math import sqrt
from time import time
import matplotlib.ticker as tkr
from scipy import stats
import math
import keras
from keras.models import Sequential
from keras.callbacks import EarlyStopping
from sklearn.metrics import mean_absolute_error

In [664]:
df_energy = pd.read_csv("df_final.csv", parse_dates=['time'])
df_energy.set_index('time', inplace=True)

In [665]:
train_data = df_energy.iloc[:int(len(df_energy)*0.7),:]
val_data = df_energy.iloc[int(len(df_energy)*0.7):int(len(df_energy)*0.85),:]
test_data = df_energy.iloc[int(len(df_energy)*0.85):,:]

In [666]:
train_X = train_data[train_data.columns.drop('total load actual')].values
train_y = train_data['total load actual'].values

val_X = val_data[val_data.columns.drop('total load actual')].values
val_y = val_data['total load actual'].values

test_X = test_data[test_data.columns.drop('total load actual')].values
test_y = test_data['total load actual'].values

In [667]:
train_y = train_y.reshape(-1, 1)
val_y = val_y.reshape(-1, 1)
test_y = test_y.reshape(-1, 1)

In [668]:
scaler = MinMaxScaler(feature_range=(0, 1))
train_X = scaler.fit_transform(train_X)
val_X = scaler.transform(val_X)
test_X = scaler.transform(test_X)

train_y = scaler.fit_transform(train_y)
val_y = scaler.transform(val_y)
test_y = scaler.transform(test_y)

In [669]:
pca = PCA(n_components=5)
train_X = pca.fit_transform(train_X)
val_X = pca.transform(val_X)
test_X = pca.transform(test_X)

In [670]:
params = {
    'n_estimators': 1000,
    'max_depth': 10,
    'eta': 0.1,
    'booster': 'gbtree',
    'objective': 'multi:softmax',
    'gamma': 0.1,
    'lambda': 2,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'min_child_weight': 3,
    'seed': 1000,
    'nthread': 4,
    'objective': 'reg:squarederror',
    'eval_metric': 'rmse'
}

In [671]:
xgb_model = xgb.XGBRegressor(**params)

xgb_model.fit(train_X, train_y, eval_set=[(val_X, val_y)])

[0]	validation_0-rmse:0.20165
[1]	validation_0-rmse:0.19859
[2]	validation_0-rmse:0.19670
[3]	validation_0-rmse:0.19465
[4]	validation_0-rmse:0.19279
[5]	validation_0-rmse:0.19121
[6]	validation_0-rmse:0.18987
[7]	validation_0-rmse:0.18910
[8]	validation_0-rmse:0.18895
[9]	validation_0-rmse:0.18892
[10]	validation_0-rmse:0.18875
[11]	validation_0-rmse:0.18819
[12]	validation_0-rmse:0.18806
[13]	validation_0-rmse:0.18773
[14]	validation_0-rmse:0.18749
[15]	validation_0-rmse:0.18767
[16]	validation_0-rmse:0.18811
[17]	validation_0-rmse:0.18804
[18]	validation_0-rmse:0.18805
[19]	validation_0-rmse:0.18824
[20]	validation_0-rmse:0.18858
[21]	validation_0-rmse:0.18881
[22]	validation_0-rmse:0.18912
[23]	validation_0-rmse:0.18906
[24]	validation_0-rmse:0.18919
[25]	validation_0-rmse:0.18957
[26]	validation_0-rmse:0.19002
[27]	validation_0-rmse:0.19037
[28]	validation_0-rmse:0.19050
[29]	validation_0-rmse:0.19053
[30]	validation_0-rmse:0.19070
[31]	validation_0-rmse:0.19071
[32]	validation_0-

[260]	validation_0-rmse:0.19754
[261]	validation_0-rmse:0.19760
[262]	validation_0-rmse:0.19766
[263]	validation_0-rmse:0.19768
[264]	validation_0-rmse:0.19771
[265]	validation_0-rmse:0.19774
[266]	validation_0-rmse:0.19777
[267]	validation_0-rmse:0.19779
[268]	validation_0-rmse:0.19775
[269]	validation_0-rmse:0.19778
[270]	validation_0-rmse:0.19777
[271]	validation_0-rmse:0.19775
[272]	validation_0-rmse:0.19771
[273]	validation_0-rmse:0.19770
[274]	validation_0-rmse:0.19772
[275]	validation_0-rmse:0.19773
[276]	validation_0-rmse:0.19779
[277]	validation_0-rmse:0.19780
[278]	validation_0-rmse:0.19780
[279]	validation_0-rmse:0.19779
[280]	validation_0-rmse:0.19787
[281]	validation_0-rmse:0.19788
[282]	validation_0-rmse:0.19795
[283]	validation_0-rmse:0.19791
[284]	validation_0-rmse:0.19787
[285]	validation_0-rmse:0.19784
[286]	validation_0-rmse:0.19783
[287]	validation_0-rmse:0.19781
[288]	validation_0-rmse:0.19783
[289]	validation_0-rmse:0.19786
[290]	validation_0-rmse:0.19783
[291]	va

[517]	validation_0-rmse:0.19914
[518]	validation_0-rmse:0.19914
[519]	validation_0-rmse:0.19913
[520]	validation_0-rmse:0.19917
[521]	validation_0-rmse:0.19913
[522]	validation_0-rmse:0.19915
[523]	validation_0-rmse:0.19917
[524]	validation_0-rmse:0.19916
[525]	validation_0-rmse:0.19917
[526]	validation_0-rmse:0.19917
[527]	validation_0-rmse:0.19912
[528]	validation_0-rmse:0.19914
[529]	validation_0-rmse:0.19916
[530]	validation_0-rmse:0.19922
[531]	validation_0-rmse:0.19918
[532]	validation_0-rmse:0.19923
[533]	validation_0-rmse:0.19922
[534]	validation_0-rmse:0.19925
[535]	validation_0-rmse:0.19924
[536]	validation_0-rmse:0.19928
[537]	validation_0-rmse:0.19929
[538]	validation_0-rmse:0.19931
[539]	validation_0-rmse:0.19930
[540]	validation_0-rmse:0.19932
[541]	validation_0-rmse:0.19938
[542]	validation_0-rmse:0.19939
[543]	validation_0-rmse:0.19934
[544]	validation_0-rmse:0.19934
[545]	validation_0-rmse:0.19933
[546]	validation_0-rmse:0.19931
[547]	validation_0-rmse:0.19931
[548]	va

[774]	validation_0-rmse:0.19977
[775]	validation_0-rmse:0.19978
[776]	validation_0-rmse:0.19978
[777]	validation_0-rmse:0.19981
[778]	validation_0-rmse:0.19982
[779]	validation_0-rmse:0.19982
[780]	validation_0-rmse:0.19985
[781]	validation_0-rmse:0.19985
[782]	validation_0-rmse:0.19985
[783]	validation_0-rmse:0.19988
[784]	validation_0-rmse:0.19985
[785]	validation_0-rmse:0.19976
[786]	validation_0-rmse:0.19978
[787]	validation_0-rmse:0.19979
[788]	validation_0-rmse:0.19977
[789]	validation_0-rmse:0.19978
[790]	validation_0-rmse:0.19977
[791]	validation_0-rmse:0.19973
[792]	validation_0-rmse:0.19982
[793]	validation_0-rmse:0.19981
[794]	validation_0-rmse:0.19976
[795]	validation_0-rmse:0.19978
[796]	validation_0-rmse:0.19974
[797]	validation_0-rmse:0.19978
[798]	validation_0-rmse:0.19979
[799]	validation_0-rmse:0.19977
[800]	validation_0-rmse:0.19978
[801]	validation_0-rmse:0.19978
[802]	validation_0-rmse:0.19983
[803]	validation_0-rmse:0.19980
[804]	validation_0-rmse:0.19980
[805]	va

XGBRegressor(base_score=None, booster='gbtree', callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.7, early_stopping_rounds=None,
             enable_categorical=False, eta=0.1, eval_metric='rmse',
             feature_types=None, gamma=0.1, gpu_id=None, grow_policy=None,
             importance_type=None, interaction_constraints=None, lambda=2,
             learning_rate=None, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=10,
             max_leaves=None, min_child_weight=3, missing=nan,
             monotone_constraints=None, n_estimators=1000, n_jobs=None,
             nthread=4, ...)

In [672]:
y_pred = xgb_model.predict(test_X)
print("Mean Absolute Error: " + str(mean_absolute_error(y_pred, test_y)))

Mean Absolute Error: 0.1363420182749589


In [673]:
rmse = np.sqrt(mean_squared_error(test_y, y_pred))
print('RMSE of hour-ahead electricity price multivariate Stacked LSTM forecast: {}'
      .format(round(rmse, 3)))

RMSE of hour-ahead electricity price multivariate Stacked LSTM forecast: 0.17
